In [2]:
import os
import json
import base64
from datetime import datetime
from io import BytesIO
import ee
import geemap
from ipyleaflet import ImageOverlay, LayersControl
import ipywidgets as widgets
from IPython.display import display
from dotenv import load_dotenv
load_dotenv()
from leafmap import leafmap
import re
import cv2
from PIL import Image, ImageDraw, ImageFont, ImageEnhance
from datetime import datetime
import rasterio
import numpy as np
import json

# --------------------------
# Ortak Hazırlık
# --------------------------
log_output = widgets.Output(layout={'border': '1px solid gray', 'height': '200px', 'overflow': 'auto'})
logf = "log.txt"
def L(msg):
    # dosyaya yaz
    with open(logf, "a", encoding="utf-8") as f:
        f.write(msg + "\n")
    # widget’a yaz
    with log_output:
        print(msg)

def get_or_create_place_folder(region):

    # 1) Geçerli ROI’yi tekdüze JSON string’e çevir
    current_roi = json.dumps(region.getInfo(), sort_keys=True)

    # 2) place_N klasörlerini numaralarına göre sırala
    folders = [d for d in os.listdir() if re.fullmatch(r'place_\d+', d)]
    folders.sort(key=lambda x: int(re.findall(r'\d+', x)[0]))

    if folders:
        last = folders[-1]
        roi_path = os.path.join(last, 'roi.json')
        # 3) eğer roi.json varsa oku ve karşılaştır
        if os.path.exists(roi_path):
            if open(roi_path, 'r', encoding='utf-8').read() == current_roi:
                return last
        # 4) farklıysa yeni numara
        next_num = int(re.findall(r'\d+', last)[0]) + 1
    else:
        next_num = 0

    # 5) Yeni klasör oluştur ve ROI’yi kaydet
    new_folder = f'place_{next_num}'
    os.makedirs(new_folder, exist_ok=True)
    
    L(f"{new_folder} Yeni dosya oluşturuldu")
    with open(os.path.join(new_folder, 'roi.json'), 'w', encoding='utf-8') as f:
        f.write(current_roi)
    L(f"roi.json oluşturuldu")
    return new_folder

ee.Authenticate()
ee.Initialize(project='688547612335')
# Initialize the Earth Engine client

# --------------------------
# 1) Sulak Alan Analizi
# --------------------------
def sulak_alan_analizi(_):
    L("Sulak alan analizi başladı")
    import os, re, ee, geemap, rasterio, numpy as np
    from PIL import Image, ImageDraw, ImageFont
    from datetime import datetime
    
    # 1) KAÇ YIL GERİ?
    year_count = 40
    today = datetime.today()
    end_year = today.year - 1
    start_year = end_year - year_count + 1
    all_years = list(range(start_year, end_year + 1))
    
    # 2) ROI’Yİ AL
    
    region = m.user_roi
    if region is None:
        raise ValueError("Önce haritada ROI seç ve kodu tekrar çalıştır!")
    
    place_folder = get_or_create_place_folder(m.user_roi)
    
    out_gif_path = os.path.join(place_folder,"sulak_alan_timelapse.gif")
            
    # 4) GIF oluşturma
    m.add_landsat_ts_gif(
        roi=m.user_roi,
        label="sulak_alan_timelapse",
        start_year=start_year,
        end_year=end_year,
        bands=["Red", "Green", "Blue"],
        frames_per_second=3,
        apply_fmask=True,
        nd_bands=["Green", "SWIR1"],
        nd_threshold=0,
        font_color="white",
        font_size=24,
        progress_bar_color="blue",
        out_gif=out_gif_path,
    )
    
    L(f"GIF Oluşturuldu ve haritaya eklendi")
    L("Sulak alan analizi tamamlandı\n\n")

# --------------------------
# 2) Kaliteli RGB Zaman Serisi
# --------------------------
def kaliteli_rgb(_):
    L("Kaliteli RGB zaman serisi başlatıldı")
    import os, re, ee, geemap, rasterio, numpy as np
    from PIL import Image, ImageDraw, ImageFont
    from datetime import datetime
    
    # 1) KAÇ YIL GERİ?
    year_count = 8
    today = datetime.today()
    end_year = today.year - 1
    start_year = end_year - year_count + 1
    all_years = list(range(start_year, end_year + 1))
    
    # 2) ROI’Yİ AL
    region = m.user_roi
    if region is None:
        raise ValueError("Önce haritada ROI seç ve kodu tekrar çalıştır!")
    
    place_folder = get_or_create_place_folder(m.user_roi)
    tif_dir  = os.path.join(place_folder, "tif_dosyaları")
    png_dir  = os.path.join(place_folder, "png_dosyaları")
    os.makedirs(tif_dir, exist_ok=True)
    os.makedirs(png_dir, exist_ok=True)
    
    L(f"{start_year}-{end_year} arası {len(all_years)} yıl için başladı.")
    L(f"Çıktılar {place_folder} içinde.")
    
    # 5) METADATA BULMA: Yalnız Sentinel-2 & max %20 bulut
    def find_meta(year):
        coll = (
            ee.ImageCollection('COPERNICUS/S2_SR')
              .filterBounds(region)
              .filterDate(f"{year}-01-01", f"{year}-06-19")
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
        )
        try:
            cnt = coll.size().getInfo()
        except:
            cnt = 0
        if cnt > 0:
            return {
                "name": "sentinel",
                "col": coll,
                "bands": ['B4', 'B3', 'B2'],
                "scale": 10,
                "thr": 20
            }
        return None
    
    # 6) İNDİRME & GEÇERLİ YILLAR
    valid = []
    for y in all_years:
        m_meta = find_meta(y)
        if not m_meta:
            L(f"{y}: veri yok veya %20 altında bulutlu → atlandı.")
            continue
    
        fn = f"{m_meta['name']}_{y}_c{m_meta['thr']}.tif"
        tp = os.path.join(tif_dir, fn)
        if os.path.exists(tp) and os.path.getsize(tp) > 1024:
            L(f"{y}: zaten var → {fn}")
            valid.append((y, tp, m_meta['name']))
            continue
    
        try:
            img = m_meta["col"].median().clip(region).select(m_meta["bands"])
            geemap.ee_export_image(
                img,
                filename=tp,
                scale=m_meta["scale"],
                region=region,
                file_per_band=False
            )
            if os.path.exists(tp) and os.path.getsize(tp) > 1024:
                L(f"{y}: indirildi → {fn}")
                valid.append((y, tp, m_meta['name']))
            else:
                L(f"UYARI: {y}: bozuk veya eksik .tif.")
        except Exception as e:
            L(f"UYARI: {y}: .tif indirilemedi: {e}")
    
    if not valid:
        L("Hiç veri indirilemedi, çıkılıyor.")
        raise SystemExit
    
    
    # 7) PNG OLUŞTUR & GÖRÜNTÜ İYİLEŞTİRME
    from PIL import ImageEnhance
    pngs = []
    for y, tp, sn in valid:
        pngp = tp.replace(".tif", ".png").replace("tif_dosyaları", "png_dosyaları")
        try:
            # raster → numpy
            with rasterio.open(tp) as src:
                arr = src.read([1, 2, 3])
                arr = np.transpose(arr, (1, 2, 0))
                mn, mx = arr.min(), arr.max()
                norm = ((arr - mn) / (mx - mn) * 255).astype(np.uint8) if mx > mn else np.zeros_like(arr, np.uint8)
    
            # numpy → PIL image
            im = Image.fromarray(norm)
    
            # --- MAKYAJ: doygunluk, kontrast, parlaklık ---
            enhancer = ImageEnhance.Color(im)
            im = enhancer.enhance(1.5)
    
            enhancer = ImageEnhance.Contrast(im)
            im = enhancer.enhance(1.3)
    
            enhancer = ImageEnhance.Brightness(im)
            im = enhancer.enhance(1.1)
            # --- makyaj sonu ---
    
            # metin ekle
            draw = ImageDraw.Draw(im)
            try:
                fnt = ImageFont.truetype("arial.ttf", 15)
            except:
                fnt = ImageFont.load_default()
            draw.text((20, 20), f"{y}", (255, 255, 255), font=fnt)
    
            # kaydet ve listeye ekle
            im.save(pngp)
            pngs.append(pngp)
            L(f"{y}: PNG yapıldı → {os.path.basename(pngp)}")
    
        except Exception as e:
            L(f"UYARI: PNG yapılamadı {os.path.basename(tp)}: {e}")
    
    
    # 8) GIF OLUŞTUR
    L("GIF hazırlanıyor…")
    frames = [Image.open(p) for p in pngs if os.path.exists(p)]
    gifp = os.path.join(place_folder, "Kaliteli_RGB_timelapse.gif")
    if frames:
        frames[0].save(
            gifp,
            save_all=True,
            append_images=frames[1:],
            duration=800,
            loop=0
        )
        L(f"GIF oluşturuldu → {os.path.basename(gifp)}")
    else:
        L("UYARI: PNG bulunamadı, GIF oluşturulamadı.")
    # 9) HARİTAYA OVERLAY
    import base64
    from ipyleaflet import ImageOverlay, LayersControl
    
    coords = region.bounds().getInfo()['coordinates'][0]
    lons = [pt[0] for pt in coords]
    lats = [pt[1] for pt in coords]
    bounds = [[min(lats), min(lons)], [max(lats), max(lons)]]
    
    with open(gifp, "rb") as f:
        data = f.read()
    data_uri = "data:image/gif;base64," + base64.b64encode(data).decode("utf-8")
    
    gif_layer = ImageOverlay(
        url=data_uri,
        bounds=bounds,
        name="Kaliteli_RGB_timelapse_GIF",
        opacity=1
    )
    m.add_layer(gif_layer)
    L("GIF haritaya eklendi")
    L("TÜM İŞLEM TAMAMLANDI ✔️")

    L("Kaliteli RGB zaman serisi tamamlandı\n\n")

# --------------------------
# 3) Şehirleşme Analizi (NDBI)
# --------------------------
def sehirlesme_analizi(_):
    L("Şehirleşme analizi başlatıldı")
    import requests           # ← add this
    from io import BytesIO    # make sure BytesIO is in scope
    # 1) KAÇ YIL GERİ?
    year_count = 8
    today = datetime.today()
    end_year = today.year -1
    start_year = end_year - year_count + 1
    
    
    # 2) ROI’Yİ AL
    region = m.user_roi
    if region is None:
        raise ValueError("Önce haritada ROI seç ve kodu tekrar çalıştır!")
    
    place_folder = get_or_create_place_folder(m.user_roi)
    
            
    # 1) Gerekli klasörleri hazırla
    png_nd_dir = os.path.join(place_folder, "png_yapısal")
    os.makedirs(png_nd_dir, exist_ok=True)
    
    
    # 3) Her yıl için NDBI kompozitini hesapla, PNG’ye dönüştür, makyaj yap ve frame listesine ekle
    frames = []
    for year in range(start_year, end_year + 1):
        # (a) NDBI = (SWIR1 - NIR) / (SWIR1 + NIR)
        ndbi = (
            ee.ImageCollection('COPERNICUS/S2_SR')
              .filterBounds(region)
              .filterDate(f'{year}-01-01', f'{year}-06-19')
              .map(lambda img: img.normalizedDifference(['B11','B8']).rename('NDBI'))
              .median()
        )
    
        # (b) Görselleştirme: açık gri → koyu kahverengi
        ndbi_vis = {
            'min': -0.2,
            'max': 0.5,
            'palette': [
                'd9d9d9',  # -0.2 → açık gri
                'bfbfbf',  # 0.0
                'a64d00',  # 0.2 → düşük yapılaşma
                '8c3300',  # 0.4
                '663300'   # 0.5 → yoğun beton
            ]
        }
        vis = ndbi.visualize(**ndbi_vis)
    
        # (c) Thumbnail URL al & indir
        thumb_url = vis.getThumbURL({
            'region': region,
            'dimensions': 512,
            'format': 'png'
        })
        resp = requests.get(thumb_url)
        im = Image.open(BytesIO(resp.content)).convert('RGBA')
    
        # (d) Görüntü makyajı: doygunluk, kontrast, parlaklık
        enh = ImageEnhance.Color(im)
        im = enh.enhance(1.8)
        enh = ImageEnhance.Contrast(im)
        im = enh.enhance(1.4)
        enh = ImageEnhance.Brightness(im)
        im = enh.enhance(1.05)
    
        # (e) Yıl etiketini ekle
        draw = ImageDraw.Draw(im)
        try:
            font = ImageFont.truetype("arial.ttf", 20)
        except:
            font = ImageFont.load_default()
        draw.text((10, 10), str(year), (255, 255, 255), font=font)
    
        # (f) PNG olarak kaydet
        png_path = os.path.join(png_nd_dir, f"ndbi_{year}.png")
        im.save(png_path)
        frames.append(im)
        L(f"{year}: NDBI PNG kaydedildi → {os.path.basename(png_path)}")
    
    # 4) NDBI timelapse GIF’e dönüştür ve kaydet
    gif_path = os.path.join(place_folder, "nd_yapısal_timelapse.gif")
    if frames:
        frames[0].save(
            gif_path,
            save_all=True,
            append_images=frames[1:],
            duration=800,
            loop=0
        )
        L(f"GIF oluşturuldu → {os.path.basename(gif_path)}")
    else:
        L("UYARI: Hiç PNG bulunamadı, GIF oluşturulamadı.")
    
    # 5) GIF’i haritaya overlay olarak ekle
    coords = region.bounds().getInfo()['coordinates'][0]
    lons = [pt[0] for pt in coords]
    lats = [pt[1] for pt in coords]
    bounds = [[min(lats), min(lons)], [max(lats), max(lons)]]
    
    with open(gif_path, "rb") as f:
        data_uri = "data:image/gif;base64," + base64.b64encode(f.read()).decode("utf-8")
    
    gif_layer = ImageOverlay(
        url=data_uri,
        bounds=bounds,
        name="NDBI yapısal Timelapse",
        opacity=1
    )
    m.add_layer(gif_layer)
    
    L("NDBI yapısal timelapse haritaya eklendi ✔️")
    L("Şehirleşme analizi tamamlandı\n\n")

# --------------------------
# 4) Yeşillik Analizi (NDVI)
# --------------------------
def yesillik_analizi(_):
    import requests           # ← add this
    from io import BytesIO    # make sure BytesIO is in scope
    L("\n\nYeşillik analizi başlatıldı")
    # 1) KAÇ YIL GERİ?
    year_count = 8
    today = datetime.today()
    end_year = today.year
    start_year = end_year - year_count
    
    # 2) ROI’Yİ AL
    region = m.user_roi
    if region is None:
        raise ValueError("Önce haritada ROI seç ve kodu tekrar çalıştır!")
    
    place_folder = get_or_create_place_folder(m.user_roi)
            
    # 1) Gerekli klasörleri hazırla
    png_nd_dir = os.path.join(place_folder, "png_yesil_alan")
    os.makedirs(png_nd_dir, exist_ok=True)
    
    
    # 3) Her yıl için NDVI kompozitini hesapla, PNG’ye dönüştür, makyaj yap ve frame listesine ekle
    frames = []
    for year in range(start_year, end_year + 1):
        # (a) NDVI = (B8 - B4) / (B8 + B4)
        ndvi = (
            ee.ImageCollection('COPERNICUS/S2_SR')
              .filterBounds(region)
              .filterDate(f'{year}-01-01', f'{year}-12-31')
              .map(lambda img: img.normalizedDifference(['B8','B4']).rename('NDVI'))
              .median()
        )
    
        # (b) Görselleştirme: koyu yeşilden parlak yeşile
        nd_vis = {
            'min': 0.0,
            'max': 1.0,
            'palette': [
                '003300',  # 0.0 → çok koyu yeşil
                '006600',  # 0.2
                '009900',  # 0.4
                '00FF00'   # 1.0 → parlak yeşil
            ]
        }
        vis = ndvi.visualize(**nd_vis)
    
        # (c) Thumbnail URL al & indir
        thumb_url = vis.getThumbURL({
            'region': region,
            'dimensions': 512,
            'format': 'png'
        })
        resp = requests.get(thumb_url)
        im = Image.open(BytesIO(resp.content)).convert('RGBA')
    
        # (d) Görüntü makyajı: doygunluk, kontrast, parlaklık
        enh = ImageEnhance.Color(im)
        im = enh.enhance(2.0)
        enh = ImageEnhance.Contrast(im)
        im = enh.enhance(1.5)
        enh = ImageEnhance.Brightness(im)
        im = enh.enhance(1.1)
    
        # (e) Yıl etiketini ekle
        draw = ImageDraw.Draw(im)
        try:
            font = ImageFont.truetype("arial.ttf", 20)
        except:
            font = ImageFont.load_default()
        draw.text((10, 10), str(year), (255, 255, 255), font=font)
    
        # (f) PNG olarak kaydet
        png_path = os.path.join(png_nd_dir, f"ndvi_{year}.png")
        im.save(png_path)
        frames.append(im)
        L(f"{year}: NDVI PNG kaydedildi → {os.path.basename(png_path)}")
    
    # 4) NDVI timelapse GIF’e dönüştür ve kaydet
    gif_path = os.path.join(place_folder, "nd_yesil_alan_timelapse.gif")
    if frames:
        frames[0].save(
            gif_path,
            save_all=True,
            append_images=frames[1:],
            duration=800,
            loop=0
        )
        L(f"GIF oluşturuldu → {os.path.basename(gif_path)}")
    else:
        L("UYARI: Hiç PNG bulunamadı, GIF oluşturulamadı.")
    
    # 5) GIF’i haritaya overlay olarak ekle
    coords = region.bounds().getInfo()['coordinates'][0]
    lons = [pt[0] for pt in coords]
    lats = [pt[1] for pt in coords]
    bounds = [[min(lats), min(lons)], [max(lats), max(lons)]]
    
    with open(gif_path, "rb") as f:
        data_uri = "data:image/gif;base64," + base64.b64encode(f.read()).decode("utf-8")
    
    gif_layer = ImageOverlay(
        url=data_uri,
        bounds=bounds,
        name="NDVI yesillik Timelapse",
        opacity=1
    )
    m.add_layer(gif_layer)
    
    L("NDVI yeşillik timelapse haritaya eklendi ✔️")
    L("Yeşillik analizi tamamlandı\n\n")

# --------------------------
# 5) Place Yükle
# --------------------------

# Tüm place_* klasörlerini, numaralarına göre sırala
places = []
for d in os.listdir():
    m = re.match(r'^(place_(\d+).*)$', d)
    if m and os.path.isdir(d):
        # m.group(1) = tüm isim (örn. "place_4(istanbul, sarıyer)")
        # m.group(2) = sadece sayı ("4")
        places.append((int(m.group(2)), m.group(1)))
# sayıya göre sırala, sonra sadece isimleri al
places = [name for _, name in sorted(places, key=lambda x: x[0])]

# Dropdown ve buton
dropdown = widgets.Dropdown(options=places, description="Place:")
btn_load = widgets.Button(description="Place Yükle", button_style="info")

def load_place(_):
    sel = dropdown.value  # örn. "place_4(istanbul, sarıyer)"
    # ROI'yi oku
    roi = json.load(open(os.path.join(sel, "roi.json"), "r", encoding="utf-8"))
    region2 = ee.Geometry(roi)
    # bounds
    coords = region2.bounds().getInfo()['coordinates'][0]
    lons = [c[0] for c in coords]; lats = [c[1] for c in coords]
    bds = [[min(lats), min(lons)], [max(lats), max(lons)]]
    # klasördeki tüm .gif'leri ekle
    for fn in sorted(os.listdir(sel)):
        if fn.lower().endswith(".gif"):
            path = os.path.join(sel, fn)
            uri = "data:image/gif;base64," + base64.b64encode(open(path, "rb").read()).decode()
            layer = ImageOverlay(url=uri, bounds=bds, name=fn, opacity=0.8)
            m.add_layer(layer)
    L(f"{sel} içindeki tüm GIF’ler haritaya eklendi.")

btn_load.on_click(load_place)

# --------------------------
# Panel Oluştur & Göster
# --------------------------
btn1 = widgets.Button(description="1) Sulak Alan",   button_style="primary")
btn2 = widgets.Button(description="2) Kaliteli RGB", button_style="warning")
btn3 = widgets.Button(description="3) Şehirleşme",  button_style="")
btn4 = widgets.Button(description="4) Yeşillik",    button_style="success")

btn1.on_click(sulak_alan_analizi)
btn2.on_click(kaliteli_rgb)
btn3.on_click(sehirlesme_analizi)
btn4.on_click(yesillik_analizi)

panel = widgets.VBox([
    widgets.HBox([btn1, btn2, btn3, btn4]),
    widgets.HBox([dropdown, btn_load])
])

m = geemap.Map(
    center=(37.5, 35.5),  # Center of Turkey
    zoom=6,
    height='650px',
    width='100%',
)
m.add_basemap('SATELLITE')  # Esri World Imagery

display(panel, m, log_output)

Map(center=[37.5, 35.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

Output(layout=Layout(border_bottom='1px solid gray', border_left='1px solid gray', border_right='1px solid gra…